In [83]:
import json
from xml.etree.ElementTree import Element, SubElement, Comment
from xml.etree import ElementTree
from xml.dom import minidom

In [84]:
with open("icd11.json") as fp:
    data = json.load(fp)

In [85]:
data

{'count': 10448,
 'captured': ['01',
  '02',
  '03',
  '04',
  '05',
  '06',
  '08',
  '09',
  '10',
  '11',
  '12',
  '13',
  '14'],
 'data': {'1A00': {'title': '1A00 Cholera',
   'theCode': '1A00',
   'chapter': '01'},
  '1A01': {'title': '1A01 Intestinal infection due to other Vibrio',
   'theCode': '1A01',
   'chapter': '01'},
  '1A02': {'title': '1A02 Intestinal infections due to Shigella',
   'theCode': '1A02',
   'chapter': '01'},
  '1A03': {'title': '1A03 Intestinal infections due to Escherichia coli',
   'theCode': '1A03',
   'chapter': '01'},
  '1A03.0': {'title': '1A03.0 Enteropathogenic Escherichia coli infection',
   'theCode': '1A03.0',
   'chapter': '01'},
  '1A03.1': {'title': '1A03.1 Enterotoxigenic Escherichia coli infection',
   'theCode': '1A03.1',
   'chapter': '01'},
  '1A03.2': {'title': '1A03.2 Enteroinvasive Escherichia coli infection',
   'theCode': '1A03.2',
   'chapter': '01'},
  '1A03.3': {'title': '1A03.3 Enterohaemorrhagic Escherichia coli infection',
   

In [45]:
def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

In [46]:
datum = data["1D01.Y"]
datum

{'id': 'http://id.who.int/icd/release/11/2019-04/mms/121670633/other',
 'title': 'Other specified infectious meningitis not elsewhere classified',
 'stemId': 'http://id.who.int/icd/release/11/2019-04/mms/121670633/other',
 'isLeaf': True,
 'postcoordinationAvailability': 2,
 'hasCodingNote': False,
 'propertiesTruncated': False,
 'isResidualOther': True,
 'isResidualUnspecified': False,
 'chapter': '01',
 'theCode': '1D01.Y',
 'score': 0.38856333,
 'titleIsASearchResult': False,
 'titleIsTopScore': False,
 'entityType': 0,
 'important': False,
 'matchingPVs': [{'propertyId': 'IndexTerm',
   'label': "<em class='found'>adhesive</em> arachnoiditis",
   'score': 0.38856333,
   'important': False}],
 'descendants': []}

In [69]:
def slugit(text):
    return text.replace('.', '-,-', 1).replace('.', '-').replace('-,-', '.')

In [70]:
def build_data_xml(data):
    tryton_el = Element('tryton')
    data_el = SubElement(tryton_el, 'data')
    for key, datum in data.items():
        record_el = SubElement(data_el, 'record')
        record_el.set('model', 'gnuhealth.pathology')
        record_el.set('id', slugit("{}".format(datum['theCode'])))


        # data_field_keys = [
        #     'id',
        #     'title',
        #     'stemId',
        #     'theCode',
        # ]
        field_name_el = SubElement(record_el, 'field')
        field_name_el.set('name', 'name')
        field_name_el.text = datum['title']

        field_code_el = SubElement(record_el, 'field')
        field_code_el.set('name', 'code')
        field_code_el.text = datum['theCode']

        field_category_el = SubElement(record_el, 'field')
        field_category_el.set('name', 'category')
        field_category_el.set('ref', slugit('icd11cat{}'.format(datum['chapter'])) )

        field_title_el = SubElement(record_el, 'field')
        field_title_el.set('name', 'active')
        field_title_el.set('eval', 'True')

        field_classifier_el = SubElement(record_el, 'field')
        field_classifier_el.set('name', 'classifier')
        field_classifier_el.text = 'ICD11'
    return tryton_el

In [71]:
def build_categories_xml(data):
    tryton_el = Element('tryton')
    data_el = SubElement(tryton_el, 'data')
    for key, text in data.items():
        record_el = SubElement(data_el, 'record')
        record_el.set('model', 'gnuhealth.pathology.category')
        record_el.set('id', slugit("icd11cat{}".format(key)))

        field_name_el = SubElement(record_el, 'field')
        field_name_el.set('name', 'name')
        field_name_el.text = text

        field_classifier_el = SubElement(record_el, 'field')
        field_classifier_el.set('name', 'classifier')
        field_classifier_el.text = 'ICD11'
    return tryton_el

In [72]:
xml_data = build_data_xml(data)
pretty_xml_data = prettify(xml_data)
print(pretty_xml_data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [73]:
tree = ElementTree.ElementTree(ElementTree.fromstring(pretty_xml_data))
tree.write('icd11.xml',encoding="UTF-8",xml_declaration=True)

In [74]:
categories = { '10': '10 Diseases of the ear or mastoid process',
  '11': '11 Diseases of the circulatory system',
  '12': '12 Diseases of the respiratory system',
  '13': '13 Diseases of the digestive system',
  '14': '14 Diseases of the skin',
  '15': '15 Diseases of the musculoskeletal system or connective tissue',
  '16': '16 Diseases of the genitourinary system',
  '17': '17 Conditions related to sexual health',
  '18': '18 Pregnancy, childbirth or the puerperium',
  '19': '19 Certain conditions originating in the perinatal period',
  '20': '20 Developmental anomalies',
  '21': '21 Symptoms, signs or clinical findings, not elsewhere classified',
  '22': '22 Injury, poisoning or certain other consequences of external causes',
  '23': '23 External causes of morbidity or mortality',
  '24': '24 Factors influencing health status or contact with health services',
  '25': '25 Codes for special purposes',
  '26': '26 Supplementary Chapter Traditional Medicine Conditions - Module I',
  '01': '01 Certain infectious or parasitic diseases',
  '02': '02 Neoplasms',
  '03': '03 Diseases of the blood or blood-forming organs',
  '04': '04 Diseases of the immune system',
  '05': '05 Endocrine, nutritional or metabolic diseases',
  '06': '06 Mental, behavioural or neurodevelopmental disorders',
  '07': '07 Sleep-wake disorders',
  '08': '08 Diseases of the nervous system',
  '09': '09 Diseases of the visual system',
  'V': 'V Supplementary section for functioning assessment',
  'X': 'X Extension Codes' }

In [75]:
xml_category = build_categories_xml(categories)
pretty_xml_category = prettify(xml_category)
print(pretty_xml_category)


<?xml version="1.0" ?>
<tryton>
  <data>
    <record id="icd11cat10" model="gnuhealth.pathology.category">
      <field name="name">10 Diseases of the ear or mastoid process</field>
      <field name="classifier">ICD11</field>
    </record>
    <record id="icd11cat11" model="gnuhealth.pathology.category">
      <field name="name">11 Diseases of the circulatory system</field>
      <field name="classifier">ICD11</field>
    </record>
    <record id="icd11cat12" model="gnuhealth.pathology.category">
      <field name="name">12 Diseases of the respiratory system</field>
      <field name="classifier">ICD11</field>
    </record>
    <record id="icd11cat13" model="gnuhealth.pathology.category">
      <field name="name">13 Diseases of the digestive system</field>
      <field name="classifier">ICD11</field>
    </record>
    <record id="icd11cat14" model="gnuhealth.pathology.category">
      <field name="name">14 Diseases of the skin</field>
      <field name="classifier">ICD11</field>
    <

In [76]:
tree = ElementTree.ElementTree(ElementTree.fromstring(pretty_xml_category))
tree.write('icd11_disease_categories.xml',encoding="UTF-8",xml_declaration=True)

In [77]:
strs = 'a.b.c.d'
count = 1
strs = strs.replace('.', '-,-', count).replace('.', '-').replace('-,-', '.')
strs

'a.b-c-d'